In [1]:
from pathlib import Path
from typing import List, Dict
from collections import defaultdict
from subprocess import run


def find_subjects(base_path: Path, prefix: str = "OAS2_") -> List[Path]:
    return sorted([
        p for p in base_path.iterdir() if p.is_dir() and p.name.startswith(prefix)
    ])


def group_visits_by_subject(subject_dirs: List[Path]) -> Dict[str, List[Path]]:
    grouped = defaultdict(list)
    for path in subject_dirs:
        parts = path.name.split('_')
        if len(parts) >= 3:
            subject_id = parts[1]
            grouped[subject_id].append(path)
    return grouped


def get_qualified_subjects(subject_groups: Dict[str, List[Path]]) -> List[str]:
    qualified = []
    for subject_id, visits in subject_groups.items():
        scan_types = {v.name.split('_')[-1] for v in visits}
        if len(scan_types) >= 3 and {"MR1", "MR3"}.issubset(scan_types):
            qualified.append(subject_id)
    return sorted(qualified)


def run_fastsurfer_segmentation(setup_dir, image, subject_id, output_dir):
    setup_dir = Path(setup_dir)
    output_dir = Path(output_dir)

    command = f"""
{setup_dir}/run_fastsurfer.sh \
--t1 {image} \
--sd {output_dir} \
--sid {subject_id} \
--seg_only \
--py python3 \
--no_biasfield \
--no_cereb \
--no_hypothal \
--allow_root
"""
    print(f"Running FastSurfer for subject: {subject_id}")
    run(command, shell=True, check=True)


def process_subject_range(
    setup_dir: str,
    output_dir: str,
    data_dir: str,
    start: int,
    end: int
):
    data_dir = Path(data_dir)
    setup_dir = Path(setup_dir)
    output_dir = Path(output_dir)

    subject_visit_dirs = find_subjects(data_dir)
    subject_groups = group_visits_by_subject(subject_visit_dirs)
    qualified_subjects = get_qualified_subjects(subject_groups)

    # Print full list of qualified subjects with indices
    print("\n=== Qualified Subjects (with MR1 and MR3, at least 3 scans total) ===")
    for idx, sid in enumerate(qualified_subjects):
        print(f"[{idx}] OAS2_{sid}")
    print("=====================================================================")

    # Clamp range
    end = min(end, len(qualified_subjects))
    assigned_subjects = qualified_subjects[start:end]

    print(f"\n>>> Processing subjects {start} to {end - 1}:")
    for idx, sid in enumerate(assigned_subjects, start=start):
        print(f"  - [{idx}] OAS2_{sid}")

    for subject_id in assigned_subjects:
        visits = subject_groups[subject_id]
        for visit in visits:
            if visit.name.endswith("MR1") or visit.name.endswith("MR3"):
                t1_image_path = visit / "RAW/mpr-1.nifti.img"
                if not t1_image_path.exists():
                    print(f"Missing T1 image for {visit.name}")
                    continue
                run_fastsurfer_segmentation(setup_dir, t1_image_path, visit.name, output_dir)


In [2]:
pip install yacs

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Person 1 does the first 5 subjects, with start=0, and end=5
# PLEASE update paths and start and end vals for your machine!
process_subject_range(
    setup_dir="/home/jovyan/FastSurfer",
    output_dir="/home/jovyan/fastsurfer_outputs",
    data_dir="/home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1",
    start=9,
    end=13
)



=== Qualified Subjects (with MR1 and MR3, at least 3 scans total) ===
[0] OAS2_0002
[1] OAS2_0005
[2] OAS2_0007
[3] OAS2_0012
[4] OAS2_0013
[5] OAS2_0017
[6] OAS2_0018
[7] OAS2_0020
[8] OAS2_0027
[9] OAS2_0031
[10] OAS2_0034
[11] OAS2_0036
[12] OAS2_0037
[13] OAS2_0040
[14] OAS2_0041
[15] OAS2_0044
[16] OAS2_0048
[17] OAS2_0049
[18] OAS2_0051
[19] OAS2_0057
[20] OAS2_0058
[21] OAS2_0061
[22] OAS2_0062
[23] OAS2_0064
[24] OAS2_0067
[25] OAS2_0070
[26] OAS2_0073
[27] OAS2_0076
[28] OAS2_0078
[29] OAS2_0079
[30] OAS2_0080
[31] OAS2_0090
[32] OAS2_0095

>>> Processing subjects 9 to 12:
  - [9] OAS2_0031
  - [10] OAS2_0034
  - [11] OAS2_0036
  - [12] OAS2_0037
Running FastSurfer for subject: OAS2_0031_MR1
Setting ENV variable FASTSURFER_HOME to script directory /home/jovyan/FastSurfer. 
Change via environment to location of your choice if this is undesired (export FASTSURFER_HOME=/dir/to/FastSurfer)
time command failing, not using time...
Start of the log for a new run_fastsurfer.sh invoca

  0%|          | 0/256 [00:00<?, ?batch/s]

[INFO: inference.py:  408]: Inference on 256 batches for coronal successful
[INFO: inference.py:  469]: Coronal inference on /home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1/OAS2_0031_MR1/RAW/mpr-1.nifti.img finished in 391.8765 seconds
[INFO: run_prediction.py:  400]: Run sagittal prediction
[INFO: dataset.py:   69]: Loading Sagittal with input voxelsize (1.0, 1.0)


  0%|          | 0/256 [00:00<?, ?batch/s]

[INFO: inference.py:  408]: Inference on 256 batches for sagittal successful
[INFO: inference.py:  469]: Sagittal inference on /home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1/OAS2_0031_MR1/RAW/mpr-1.nifti.img finished in 406.0017 seconds
[INFO: run_prediction.py:  400]: Run axial prediction
[INFO: dataset.py:   74]: Loading Axial with input voxelsize (1.0, 1.0)


...

[INFO: inference.py:  408]: Inference on 256 batches for coronal successful
[INFO: inference.py:  469]: Coronal inference on /home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1/OAS2_0037_MR3/RAW/mpr-1.nifti.img finished in 392.0780 seconds
[INFO: run_prediction.py:  400]: Run sagittal prediction
[INFO: dataset.py:   69]: Loading Sagittal with input voxelsize (1.0, 1.0)


  0%|          | 0/256 [00:00<?, ?batch/s]

[INFO: inference.py:  408]: Inference on 256 batches for sagittal successful
[INFO: inference.py:  469]: Sagittal inference on /home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1/OAS2_0037_MR3/RAW/mpr-1.nifti.img finished in 401.6667 seconds
[INFO: run_prediction.py:  400]: Run axial prediction
[INFO: dataset.py:   74]: Loading Axial with input voxelsize (1.0, 1.0)


100%|██████████| 256/256 [06:31<00:00,  1.53s/batch]


[INFO: inference.py:  408]: Inference on 256 batches for axial successful
[INFO: inference.py:  469]: Axial inference on /home/jovyan/shared/data/OASIS-2/OAS2_RAW_PART1/OAS2_0037_MR3/RAW/mpr-1.nifti.img finished in 391.7458 seconds
[INFO: run_prediction.py:  454]: Successfully saved image   as /home/jovyan/fastsurfer_outputs/OAS2_0037_MR3/mri/aparc.DKTatlas+aseg.deep.mgz.
[INFO: run_prediction.py:  724]: Creating brainmask based on segmentation...
Creating dilated mask ...
Frontal region special treatment:  18058
  Found 1 connected component(s)!
[INFO: run_prediction.py:  454]: Successfully saved image   as /home/jovyan/fastsurfer_outputs/OAS2_0037_MR3/mri/mask.mgz.
[INFO: run_prediction.py:  743]: Creating aseg based on segmentation...
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
[INFO: run_prediction.py:  454]: Successfully saved image   as /home/jovyan/fastsurfer_outputs/OAS2_0037_MR3/mri/aseg.auto_noCCseg.mgz.
[INFO: run_prediction.py:  762]: Running volume-based QC check o